In [117]:
import PyPDF2
import pandas as pd
import re
# Open the PDF file
pdf_file = open('Chan2014a.pdf', 'rb')

# Create a PDF reader object
pdf_reader = PyPDF2.PdfReader(pdf_file)

# Extract text from the PDF file
text = ''
# for page_num in range(pdf_reader.getNumPages()):
for page_num in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[page_num]
    text += page.extract_text()
#save the data as txt
with open("newDataType.txt", "w") as f:
    f.write(text)

# Open the text file and read the contents into a string variable
with open('newDataType.txt', 'r') as file:
    input_str = file.read()
    # Split the input string into individual lines
lines = input_str.split('\n')

# Data Extract 
-> build the Dataframes
# Categorical variables
- Metadata : [df_Metadata_paper,df_Metadata_ref]
- df_Metadata_ref : [authors, title, journal, volume, issue, page]
- df_Metadata_paper : [Author, title, keyword, abstract]
# Numerical Variables
- df_Metadata_Num : [Year]
- df_numOfRef : [num of ref]
- df_AuthorInsideCitations = [num of inside citation]


In [118]:
# DataExtraction : using pdf reader metadata


Paper_Title = pdf_reader.metadata.title
paper_Author = pdf_reader.metadata.author
info = pdf_reader.metadata
Keywords = info.get('/Keywords')

data = [Paper_Title,paper_Author,Keywords]


df_Metadata_paper = pd.DataFrame({'singles_var': data}, index=['Paper_Title','Paper_Author','Keywords'])

df_Metadata_paper

,singles_var
Paper_Title,Hiring Biases in Online Labor Markets: The Cas...
Paper_Author,Jason Chan and Jing Wang
Keywords,"Gender Stereotypes, Hiring Bias, Matching, Onl..."


In [119]:
import xml.etree.ElementTree as ET

# Open the file and read its contents
with open("2014a.pdf.tei.xml", "r", encoding="utf-8") as file:
    tei_xml = file.read()

# Parse the TEI XML
root = ET.fromstring(tei_xml)

# Find the abstract element
abstract_element = root.find('.//{http://www.tei-c.org/ns/1.0}abstract')

# Extract the text content of the abstract
abstract_text = abstract_element.find('.//{http://www.tei-c.org/ns/1.0}p').text

data = [Paper_Title,paper_Author,Keywords,abstract_text]

df_Metadata_paper = pd.DataFrame({'singles_var': data}, index=['paper_title','paper_author','keywords','abstract'])

df_Metadata_paper

,singles_var
paper_title,Hiring Biases in Online Labor Markets: The Cas...
paper_author,Jason Chan and Jing Wang
keywords,"Gender Stereotypes, Hiring Bias, Matching, Onl..."
abstract,Online labor marketplaces facilitate the effic...


build the DataFrame df_Metadata_ref

In [129]:
import pandas as pd
import xml.etree.ElementTree as ET

# Load the TEI XML file
tree = ET.parse('2014a.pdf.tei.xml')
root = tree.getroot()

# Define the XML namespace
namespace = {'tei': 'http://www.tei-c.org/ns/1.0'}

# Define a function to extract author names from the XML structure
def extract_author_names(authors):
    author_names = []
    for author in authors:
        forename_element = author.find('.//tei:forename', namespace)
        surname_element = author.find('.//tei:surname', namespace)
        if forename_element is not None and surname_element is not None:
            forename = forename_element.text
            surname = surname_element.text
            author_names.append(forename + ' ' + surname)
    return author_names

# Initialize an empty list to store the extracted data
data_list = []

# Iterate over each biblStruct element
for bibl_struct in root.findall('.//tei:biblStruct', namespace):
    title_element = bibl_struct.find('.//tei:title', namespace)
    date_element = bibl_struct.find('.//tei:date[@type="published"]', namespace)
    publisher_element = bibl_struct.find('.//tei:imprint/tei:publisher', namespace)
    bibl_scope_volume_element = bibl_struct.find('.//tei:imprint/tei:biblScope[@unit="volume"]', namespace)
    bibl_scope_issue_element = bibl_struct.find('.//tei:imprint/tei:biblScope[@unit="issue"]', namespace)
    bibl_scope_page_element = bibl_struct.find('.//tei:imprint/tei:biblScope[@unit="page"]', namespace)
    pub_place_element = bibl_struct.find('.//tei:imprint/tei:pubPlace', namespace)

    # Check if the required elements exist
    if title_element is not None and date_element is not None:
        # Extract information from the XML for the current biblStruct element
        level = title_element.attrib.get('level', '')
        title = title_element.text
        if level == 'j':
            publication_place = pub_place_element.text if pub_place_element is not None else ''
        else:
            publication_place = ''

        pages_from = ''
        pages_to = ''
        if bibl_scope_page_element is not None:
            pages_from = bibl_scope_page_element.attrib.get('from', '')
            pages_to = bibl_scope_page_element.attrib.get('to', '')

        data = {
            'Author': extract_author_names(bibl_struct.findall('.//tei:author', namespace)),
            'Title': title,
            'Publication Year': date_element.attrib.get('when', ''),
            'Publisher': publisher_element.text if publisher_element is not None else '',
            'Volume': bibl_scope_volume_element.text if bibl_scope_volume_element is not None else '',
            'Issue': bibl_scope_issue_element.text if bibl_scope_issue_element is not None else '',
            'Pages From': pages_from,
            'Pages To': pages_to,
            'Level': level,
            'Publication Place': publication_place
        }

        # Append the extracted data to the list
        data_list.append(data)

# Create a DataFrame from the extracted data
df_Metadata_ref = pd.DataFrame(data_list)

# Filter the DataFrame to include only the entries with a level of 'a' and type 'main'
df_Metadata_ref

,Author,Title,Publication Year,Publisher,Volume,Issue,Pages From,Pages To,Level,Publication Place
0,"[A Agrawal, J Horton, N Lacetera, E Lyons]",Digitization and the Contract Labor Market: A ...,2013,,,,,,m,
1,"[A Agrawal, N Lacetera, E Lyons]",Does Information Help Or Hinder Job Applicants...,2012,NBER Working Paper,,,,,m,
2,[G Akerlof],"The Market for ""Lemons"": Quality Uncertainty a...",1970,,84,3,488,500,a,
3,"[J Altonji, C Pierret]",Employer Learning and Statistical Discrimination,2001,,116,1,313,350,a,
4,"[J Andreoni, R Petrie]","Beauty, gender and stereotypes: Evidence from ...",2008,,29,1,73,93,a,
5,[K Arrow],The Theory of Discrimination,1973,Princeton University Press,,,,,a,
6,[D Autor],Wiring the Labor Market,2001,,15,1,25,40,a,
7,"[L Babcock, S Laschever]",Women Don't Ask: Negotiation and the Gender Di...,2003,Princeton University Press,,,,,m,
8,[J Bakos],Reducing Buyer Search Costs: Implications for ...,1997,,87,2,386,410,a,
9,"[M Lin, P Goes]","Does Information Really ""Unravel""? Understandi...",2012,,,,,,m,


In [143]:
# Create a new column called 'Relevance'
df_Metadata_ref['Relevance'] = 0

# Set the value of 'Relevance' to 1 for the specified lines
df_Metadata_ref.loc[[20, 21], 'Relevance'] = 1
display(df_Metadata_ref)

,Author,Title,Publication Year,Publisher,Volume,Issue,Pages From,Pages To,Level,Publication Place,Relevance
0,"[A Agrawal, J Horton, N Lacetera, E Lyons]",Digitization and the Contract Labor Market: A ...,2013,,,,,,m,,0
1,"[A Agrawal, N Lacetera, E Lyons]",Does Information Help Or Hinder Job Applicants...,2012,NBER Working Paper,,,,,m,,0
2,[G Akerlof],"The Market for ""Lemons"": Quality Uncertainty a...",1970,,84,3,488,500,a,,0
3,"[J Altonji, C Pierret]",Employer Learning and Statistical Discrimination,2001,,116,1,313,350,a,,0
4,"[J Andreoni, R Petrie]","Beauty, gender and stereotypes: Evidence from ...",2008,,29,1,73,93,a,,0
5,[K Arrow],The Theory of Discrimination,1973,Princeton University Press,,,,,a,,0
6,[D Autor],Wiring the Labor Market,2001,,15,1,25,40,a,,0
7,"[L Babcock, S Laschever]",Women Don't Ask: Negotiation and the Gender Di...,2003,Princeton University Press,,,,,m,,0
8,[J Bakos],Reducing Buyer Search Costs: Implications for ...,1997,,87,2,386,410,a,,0
9,"[M Lin, P Goes]","Does Information Really ""Unravel""? Understandi...",2012,,,,,,m,,0


In [144]:

# create DataFrame for numerical variables
df_Metadata_ref_num = df_Metadata_ref[['Relevance', 'Publication Year', 'Volume', 'Issue', 'Pages From', 'Pages To']]
df_Metadata_ref_num

,Relevance,Publication Year,Volume,Issue,Pages From,Pages To
0,0,2013,,,,
1,0,2012,,,,
2,0,1970,84,3,488,500
3,0,2001,116,1,313,350
4,0,2008,29,1,73,93
5,0,1973,,,,
6,0,2001,15,1,25,40
7,0,2003,,,,
8,0,1997,87,2,386,410
9,0,2012,,,,


In [133]:
# create DataFrame for categorical variables
df_Metadata_ref_cal = df_Metadata_ref[['Author', 'Title', 'Publisher', 'Level', 'Publication Place']]
df_Metadata_ref_cal


,Author,Title,Publisher,Level,Publication Place
0,"[A Agrawal, J Horton, N Lacetera, E Lyons]",Digitization and the Contract Labor Market: A ...,,m,
1,"[A Agrawal, N Lacetera, E Lyons]",Does Information Help Or Hinder Job Applicants...,NBER Working Paper,m,
2,[G Akerlof],"The Market for ""Lemons"": Quality Uncertainty a...",,a,
3,"[J Altonji, C Pierret]",Employer Learning and Statistical Discrimination,,a,
4,"[J Andreoni, R Petrie]","Beauty, gender and stereotypes: Evidence from ...",,a,
5,[K Arrow],The Theory of Discrimination,Princeton University Press,a,
6,[D Autor],Wiring the Labor Market,,a,
7,"[L Babcock, S Laschever]",Women Don't Ask: Negotiation and the Gender Di...,Princeton University Press,m,
8,[J Bakos],Reducing Buyer Search Costs: Implications for ...,,a,
9,"[M Lin, P Goes]","Does Information Really ""Unravel""? Understandi...",,m,


In [120]:
df_Metadata_paper.singles_var[3]

'Online labor marketplaces facilitate the efficient matching of employers and workers across geographical boundaries. The exponential growth of this nascent online phenomenon holds important social and economic implications. Despite this importance, limited effort has been devoted to understand whether potential hiring biases exist in online labor platforms and how they affect hiring outcomes. Using a novel proprietary dataset from a leading online labor platform, we investigate the impact of gender-based stereotypes on hiring outcomes. After accounting for endogeneity via a matched sample approach and quasi-experimental technique, we find evidence of a positive hiring bias towards female workers at the aggregate level. Sub-category analyses show that women are preferred in female-dominated occupations, while men are preferred in male-dominated occupations. Interestingly, women also gain an advantage in genderneutral jobs. We find that the observed hiring bias diminishes as employers g

In [121]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Initialize the stemmer
stemmer = PorterStemmer()

# Example big paragraph
paragraph = df_Metadata_paper.singles_var[3]


# Tokenize the paragraph into words
tokens = word_tokenize(paragraph)

# Apply stemming to each word
stemmed_tokens = [stemmer.stem(token) for token in tokens]

# Join the stemmed tokens back into a paragraph
stemmed_paragraph = ' '.join(stemmed_tokens)

# Apply stemming to each word
stemmed_tokens = [stemmer.stem(token) for token in tokens]

# Print the original and stemmed paragraphs
print("Original paragraph:")
print(paragraph)
print(len(paragraph))
print("\nStemmed paragraph:")
print(stemmed_paragraph)
print(len(stemmed_paragraph))
print("\nStemmed words:")
print(stemmed_tokens)
print(len(stemmed_tokens))

Original paragraph:
Online labor marketplaces facilitate the efficient matching of employers and workers across geographical boundaries. The exponential growth of this nascent online phenomenon holds important social and economic implications. Despite this importance, limited effort has been devoted to understand whether potential hiring biases exist in online labor platforms and how they affect hiring outcomes. Using a novel proprietary dataset from a leading online labor platform, we investigate the impact of gender-based stereotypes on hiring outcomes. After accounting for endogeneity via a matched sample approach and quasi-experimental technique, we find evidence of a positive hiring bias towards female workers at the aggregate level. Sub-category analyses show that women are preferred in female-dominated occupations, while men are preferred in male-dominated occupations. Interestingly, women also gain an advantage in genderneutral jobs. We find that the observed hiring bias dimini

In [122]:
abstract_stemmed = stemmed_paragraph
data = [Paper_Title,paper_Author,Keywords,abstract_text,abstract_stemmed]

df_Metadata_paper = pd.DataFrame({'singles_var': data}, index=['paper_title','paper_author','keywords','abstract','abstract_stemmed'])
df_Metadata_paper

,singles_var
paper_title,Hiring Biases in Online Labor Markets: The Cas...
paper_author,Jason Chan and Jing Wang
keywords,"Gender Stereotypes, Hiring Bias, Matching, Onl..."
abstract,Online labor marketplaces facilitate the effic...
abstract_stemmed,onlin labor marketplac facilit the effici matc...


Cleaning


In [138]:
import pandas as pd

def preprocess_dataframe(df):
    # Convert column names to lowercase and replace spaces with underscores
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    # df.columns = df.columns.lower().replace(' ', '_')

    
    # Get the list of string columns
    string_columns = list(df.dtypes[df.dtypes == 'object'].index)
    
    # Convert string columns to lowercase and replace spaces with underscores
    for col in string_columns:
        df[col] = df[col].str.lower().str.replace(' ', '_')
        df[col] = df[col].replace('[', '').replace(']', '')

        
    return df

In [139]:
preprocess_dataframe(df_Metadata_paper)
# preprocess_dataframe(df_Metadata_ref_cal)

,singles_var
paper_title,hiring_biases_in_online_labor_markets:_the_cas...
paper_author,jason_chan_and_jing_wang
keywords,"gender_stereotypes,_hiring_bias,_matching,_onl..."
abstract,online_labor_marketplaces_facilitate_the_effic...
abstract_stemmed,onlin_labor_marketplac_facilit_the_effici_matc...


In [125]:
preprocess_dataframe(df_Metadata_paper)

,singles_var
paper_title,hiring_biases_in_online_labor_markets:_the_cas...
paper_author,jason_chan_and_jing_wang
keywords,"gender_stereotypes,_hiring_bias,_matching,_onl..."
abstract,online_labor_marketplaces_facilitate_the_effic...
abstract_stemmed,onlin_labor_marketplac_facilit_the_effici_matc...


Spliting
- subdataframe : train, val and test dataframes

In [ ]:
from sklearn.model_selection import train_test_split
df_train_paper_full, df_paper_test = train_test_split(df_Metadata_ref, test_size=0.2, random_state=1)
df_paper_train, df_paper_val = train_test_split(df_train_paper_full, test_size=0.33, random_state=11)
y_paper_train = df_paper_train.relevance.values
y_paper_val = df_paper_val.relevance.values

Feature Creation
- check similarity each title with the abstract

In [106]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(paragraph, titles):
    # Preprocess the paragraph
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    paragraph = ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(paragraph.lower()) if word.isalnum() and word not in stop_words])

    # Compute TF-IDF vector for the paragraph
    vectorizer = TfidfVectorizer()
    tfidf_matrix_paragraph = vectorizer.fit_transform([paragraph])

    # Calculate cosine similarity between the paragraph and each title
    similarities = {}
    for title in titles:
        # Preprocess the title
        preprocessed_title = ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(title.lower()) if word.isalnum() and word not in stop_words])

        # Compute TF-IDF vector for the title
        tfidf_matrix_title = vectorizer.transform([preprocessed_title])

        # Calculate cosine similarity
        similarity_score = cosine_similarity(tfidf_matrix_paragraph, tfidf_matrix_title)[0][0]
        similarities[title] = round(similarity_score, 2)

    return similarities

TESTING

In [108]:
paragraph = "Online labor marketplaces facilitate the efficient matching of employers and workers across geographical boundaries. The exponential growth of this nascent online phenomenon holds important social and economic implications. Despite this importance."

titles = ["online phenomenon holds important social and economic","Digitization and the Contract Labor Market: A Research Agenda.", "Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?", "labor marketplaces facilitate the efficient matching", "Employer Learning and Statistical Discrimination"]
similarities = calculate_similarity(paragraph, titles)

for title, similarity in similarities.items():
    print(f"{title}: {similarity}")

online phenomenon holds important social and economic: 0.57
Digitization and the Contract Labor Market: A Research Agenda.: 0.2
Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?: 0.4
labor marketplaces facilitate the efficient matching: 0.45
Employer Learning and Statistical Discrimination: 0.2


In [111]:
# stemmed_abstract = df_Metadata_paper.singles_var[4]
# # similarities = calculate_similarity(stemmed_abstract, titles)

# # for title, similarity in similarities.items():
# #     print(f"{title}: {similarity}")
# stemmed_abstract

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(paragraph, titles, n=1):
    # Preprocess the paragraph
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    paragraph = ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(paragraph.lower()) if word.isalnum() and word not in stop_words])

    # Compute TF-IDF vector for the paragraph
    vectorizer = TfidfVectorizer(ngram_range=(1, n))
    tfidf_matrix_paragraph = vectorizer.fit_transform([paragraph])

    # Calculate cosine similarity between the paragraph and each title
    similarities = {}
    for title in titles:
        # Preprocess the title
        preprocessed_title = ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(title.lower()) if word.isalnum() and word not in stop_words])

        # Compute TF-IDF vector for the title
        tfidf_matrix_title = vectorizer.transform([preprocessed_title])

        # Calculate cosine similarity
        similarity_score = cosine_similarity(tfidf_matrix_paragraph, tfidf_matrix_title)[0][0]
        similarities[title] = round(similarity_score, 2)

    return similarities


In [ ]:
paragraph = "Online labor marketplaces facilitate the efficient matching of employers and workers across geographical boundaries. The exponential growth of this nascent online phenomenon holds important social and economic implications. Despite this importance."
titles = ["online phenomenon holds important social and economic","Digitization and the Contract Labor Market: A Research Agenda.", "Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?", "labor marketplaces facilitate the efficient matching", "Employer Learning and Statistical Discrimination"]
similarities = calculate_similarity(paragraph, titles, n=1)

for title, similarity in similarities.items():
    print(f"{title}: {similarity}")


online phenomenon holds important social and economic: 0.57
Digitization and the Contract Labor Market: A Research Agenda.: 0.2
Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?: 0.4
labor marketplaces facilitate the efficient matching: 0.45
Employer Learning and Statistical Discrimination: 0.2


In [ ]:
paragraph = "Online labor marketplaces facilitate the efficient matching of employers and workers across geographical boundaries. The exponential growth of this nascent online phenomenon holds important social and economic implications. Despite this importance."
titles = ["online phenomenon holds important social and economic","Digitization and the Contract Labor Market: A Research Agenda.", "Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?", "labor marketplaces facilitate the efficient matching", "Employer Learning and Statistical Discrimination"]
similarities = calculate_similarity(paragraph, titles, n=2)

for title, similarity in similarities.items():
    print(f"{title}: {similarity}")

online phenomenon holds important social and economic: 0.53
Digitization and the Contract Labor Market: A Research Agenda.: 0.15
Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?: 0.29
labor marketplaces facilitate the efficient matching: 0.44
Employer Learning and Statistical Discrimination: 0.15


In [ ]:
paragraph = "Online labor marketplaces facilitate the efficient matching of employers and workers across geographical boundaries. The exponential growth of this nascent online phenomenon holds important social and economic implications. Despite this importance."
titles = ["online phenomenon holds important social and economic","Digitization and the Contract Labor Market: A Research Agenda.", "Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?", "labor marketplaces facilitate the efficient matching", "Employer Learning and Statistical Discrimination"]
similarities = calculate_similarity(paragraph, titles, n=3)

for title, similarity in similarities.items():
    print(f"{title}: {similarity}")

online phenomenon holds important social and economic: 0.5
Digitization and the Contract Labor Market: A Research Agenda.: 0.12
Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?: 0.24
labor marketplaces facilitate the efficient matching: 0.42
Employer Learning and Statistical Discrimination: 0.12


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(titles, n=1):
    # Preprocess the titles
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    preprocessed_titles = [' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(title.lower()) if word.isalnum() and word not in stop_words]) for title in titles]

    # Compute TF-IDF vectors for the titles
    vectorizer = TfidfVectorizer(ngram_range=(1, n))
    tfidf_matrix_titles = vectorizer.fit_transform(preprocessed_titles)

    # Calculate cosine similarity between each pair of titles
    similarity_matrix = cosine_similarity(tfidf_matrix_titles)

    return similarity_matrix


In [ ]:
titles = ["online phenomenon holds important social and economic","Digitization and the Contract Labor Market: A Research Agenda.", "Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?", "labor marketplaces facilitate the efficient matching", "Employer Learning and Statistical Discrimination"]
similarity_matrix = calculate_similarity(titles, n=1)

num_titles = len(titles)
for i in range(num_titles):
    for j in range(i+1, num_titles):
        similarity = similarity_matrix[i][j]
        print(f"Similarity between '{titles[i]}' and '{titles[j]}': {similarity}")


Similarity between 'online phenomenon holds important social and economic' and 'Digitization and the Contract Labor Market: A Research Agenda.': 0.0
Similarity between 'online phenomenon holds important social and economic' and 'Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?': 0.089780355755817
Similarity between 'online phenomenon holds important social and economic' and 'labor marketplaces facilitate the efficient matching': 0.0
Similarity between 'online phenomenon holds important social and economic' and 'Employer Learning and Statistical Discrimination': 0.0
Similarity between 'Digitization and the Contract Labor Market: A Research Agenda.' and 'Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?': 0.09268890749988493
Similarity between 'Digitization and the Contract Labor Market: A Research Agenda.' and 'labor marketplaces facilitate the efficient matching': 0.13108191006788258
Similarity

In [ ]:
similarity_matrix = calculate_similarity(titles, n=2)

num_titles = len(titles)
for i in range(num_titles):
    for j in range(i+1, num_titles):
        similarity = similarity_matrix[i][j]
        print(f"Similarity between '{titles[i]}' and '{titles[j]}': {similarity}")


Similarity between 'online phenomenon holds important social and economic' and 'Digitization and the Contract Labor Market: A Research Agenda.': 0.0
Similarity between 'online phenomenon holds important social and economic' and 'Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?': 0.046621284482176
Similarity between 'online phenomenon holds important social and economic' and 'labor marketplaces facilitate the efficient matching': 0.0
Similarity between 'online phenomenon holds important social and economic' and 'Employer Learning and Statistical Discrimination': 0.0
Similarity between 'Digitization and the Contract Labor Market: A Research Agenda.' and 'Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?': 0.04740459943861528
Similarity between 'Digitization and the Contract Labor Market: A Research Agenda.' and 'labor marketplaces facilitate the efficient matching': 0.06895038822332501
Similarity

In [ ]:
similarity_matrix = calculate_similarity(titles, n=3)

num_titles = len(titles)
for i in range(num_titles):
    for j in range(i+1, num_titles):
        similarity = similarity_matrix[i][j]
        print(f"Similarity between '{titles[i]}' and '{titles[j]}': {similarity}")


Similarity between 'online phenomenon holds important social and economic' and 'Digitization and the Contract Labor Market: A Research Agenda.': 0.0
Similarity between 'online phenomenon holds important social and economic' and 'Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?': 0.03315886135963501
Similarity between 'online phenomenon holds important social and economic' and 'labor marketplaces facilitate the efficient matching': 0.0
Similarity between 'online phenomenon holds important social and economic' and 'Employer Learning and Statistical Discrimination': 0.0
Similarity between 'Digitization and the Contract Labor Market: A Research Agenda.' and 'Does Information Help Or Hinder Job Applicants from Less Developed Countries in Online Markets?': 0.03356109798460118
Similarity between 'Digitization and the Contract Labor Market: A Research Agenda.' and 'labor marketplaces facilitate the efficient matching': 0.050425373620636316
Similar

# One-hot encoding

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [115]:
# train_dict = [TODO_DF].to_dict(orient='records')
# train_dict[0]

Training using the LR and Vectorizer

In [ ]:
# dv = DictVectorizer(sparse=False)
# dv.fit(train_dict)
# X_train = dv.transform(train_dict)

# Training LR

In [116]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', random_state=1)
# model.fit(X_train, y_train)

In [ ]:
# TO SET NEW Cat
#val_dict = df_val[categorical + numerical].to_dict(orient='records')
# X_val = dv.transform(val_dict)

In [ ]:
# model.predict_proba(X_val)
# y_pred = model.predict_proba(X_val)[:, 1]

In [ ]:
# relevanz = y_pred > 0.7
# (y_val == relevanz).mean()

# use the model

In [ ]:
# X_test = dv.transform([relevanz])
# model.predict_proba(X_test)[0, 1]